# Setup


In [2]:
import sys
import subprocess
import os


def _pip_install(pkgs):
    cmd = [sys.executable, "-m", "pip", "install", "-q"] + pkgs
    subprocess.check_call(cmd)


# Install core dependencies (best-effort)
_packages = [
    "transformers>=4.36.0",
    "sentence-transformers>=2.2.2",
    "rank-bm25>=0.2.2",
    "tiktoken>=0.5.1",
    "spacy>=3.7.2",
]
for _pkg in _packages:
    try:
        _pip_install([_pkg])
    except Exception as e:
        print(f"pip install failed for {_pkg}: {e}")

# Determinism
import random
import numpy as np
import torch


def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


set_seed(42)


# Compressor Implementation


Compression notes:
- `importance_cutoff` is the fraction of lowest-surprisal tokens to remove (higher means more aggressive).
- Surprisal pruning operates at the LM token (BPE) level to preserve spacing and punctuation.
- JSON/YAML and code blocks are never pruned; they are only selected or dropped as chunks.


In [3]:
import re
import json
import math
import time
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional

import numpy as np

# Optional token counter
try:
    import tiktoken
    _tiktoken = tiktoken.get_encoding("cl100k_base")
except Exception:
    _tiktoken = None

# Lazy-loaded models
_lm_tokenizer = None
_lm_model = None
_embedder = None
_spacy_nlp = None


def get_lm():
    global _lm_tokenizer, _lm_model
    if _lm_tokenizer is None or _lm_model is None:
        from transformers import AutoTokenizer, AutoModelForCausalLM
        _lm_tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
        _lm_model = AutoModelForCausalLM.from_pretrained("distilgpt2")
        _lm_model.eval()
    return _lm_tokenizer, _lm_model


def get_embedder():
    global _embedder
    if _embedder is None:
        from sentence_transformers import SentenceTransformer
        _embedder = SentenceTransformer("all-MiniLM-L6-v2")
    return _embedder


def get_spacy_nlp():
    global _spacy_nlp
    if _spacy_nlp is None:
        try:
            import spacy
            _spacy_nlp = spacy.load("en_core_web_sm")
        except Exception:
            _spacy_nlp = None
    return _spacy_nlp


def count_tokens(text: str) -> int:
    if _tiktoken is not None:
        return len(_tiktoken.encode(text))
    try:
        tokenizer, _ = get_lm()
        return len(tokenizer.encode(text))
    except Exception:
        return max(1, len(text.split()))


def split_code_fences(text: str) -> List[Dict[str, str]]:
    pattern = re.compile(r"```.*?```", re.DOTALL)
    segments = []
    last = 0
    for m in pattern.finditer(text):
        if m.start() > last:
            segments.append({"type": "text", "text": text[last:m.start()]})
        segments.append({"type": "code", "text": text[m.start():m.end()]})
        last = m.end()
    if last < len(text):
        segments.append({"type": "text", "text": text[last:]})
    return segments


def is_json_like(text: str) -> bool:
    s = text.strip()
    if not s:
        return False
    if s[0] in "[{":
        try:
            json.loads(s)
            return True
        except Exception:
            if re.search(r"\"[^\"]+\"\s*:", s):
                return True
    return False


def is_yaml_like(text: str) -> bool:
    s = text.strip()
    if not s or s[0] in "[{":
        return False
    lines = [ln for ln in s.splitlines() if ln.strip()]
    if len(lines) < 2:
        return False
    key_like = 0
    for ln in lines:
        if re.match(r"^[\w\-\s]+:\s*", ln):
            key_like += 1
    return key_like >= max(2, len(lines) // 2)


def split_into_chunks(text: str) -> List[str]:
    parts = [p for p in re.split(r"\n\s*\n", text) if p.strip()]
    if len(parts) <= 1:
        sentences = [s for s in re.split(r"(?<=[.!?])\s+", text.strip()) if s.strip()]
        return sentences if sentences else [text]
    return parts


def extract_keywords(question: str, max_keywords: int = 10) -> List[str]:
    nlp = get_spacy_nlp()
    keywords = []
    if nlp:
        doc = nlp(question)
        for ent in doc.ents:
            keywords.append(ent.text)
        for chunk in doc.noun_chunks:
            keywords.append(chunk.text)
    else:
        tokens = re.findall(r"[A-Za-z0-9\-/_.]+", question)
        stop = {
            "the", "a", "an", "and", "or", "of", "to", "in", "for", "on", "with",
            "what", "which", "who", "when", "where", "why", "how", "is", "are", "was",
            "were", "be", "by", "from", "that", "this", "it", "as", "at"
        }
        for t in tokens:
            if t.lower() not in stop and len(t) > 2:
                keywords.append(t)
    # Normalize and dedupe
    cleaned = []
    seen = set()
    for kw in keywords:
        kw_norm = kw.strip().lower()
        if kw_norm and kw_norm not in seen:
            seen.add(kw_norm)
            cleaned.append(kw.strip())
    return cleaned[:max_keywords]


def embed_texts(texts: List[str]) -> np.ndarray:
    embedder = get_embedder()
    emb = embedder.encode(texts, normalize_embeddings=True)
    return np.asarray(emb)


def cosine_sim(a: np.ndarray, b: np.ndarray) -> float:
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-9))


def mmr_select(embeddings: np.ndarray, query_embedding: np.ndarray, token_counts: List[int],
               budget: int, mmr_lambda: float) -> List[int]:
    selected = []
    if len(embeddings) == 0 or budget <= 0:
        return selected

    sims = embeddings @ query_embedding
    remaining = set(range(len(embeddings)))
    total = 0

    while remaining:
        best_idx = None
        best_score = -1e9
        for i in sorted(remaining):
            if total + token_counts[i] > budget:
                continue
            max_sim = 0.0
            if selected:
                max_sim = max(cosine_sim(embeddings[i], embeddings[j]) for j in selected)
            score = mmr_lambda * sims[i] - (1.0 - mmr_lambda) * max_sim
            if score > best_score:
                best_score = score
                best_idx = i
        if best_idx is None:
            break
        selected.append(best_idx)
        remaining.remove(best_idx)
        total += token_counts[best_idx]
    return selected


def find_protected_spans(text: str) -> List[Tuple[int, int]]:
    spans = []

    def add_spans(pattern, flags=0):
        for m in re.finditer(pattern, text, flags):
            spans.append((m.start(), m.end()))

    # URLs
    add_spans(r"https?://\S+")
    # File paths
    add_spans(r"(?:[A-Za-z]:\\|/)[\w\-._~/:\\]+")
    # Numbers / dates / IDs
    add_spans(r"\b\d+(?:[\-/:]\d+)*\b")
    # Quoted strings
    add_spans(r"(['\"])(?:(?=(\\?))\2.)*?\1")
    # Negations and critical operators
    negations = [
        "not", "never", "no", "without", "except", "unless", "must", "mustn't",
        "don't", "do not", "can't", "cannot", "at least", "at most"
    ]
    for term in negations:
        add_spans(r"\b" + re.escape(term) + r"\b", flags=re.IGNORECASE)

    # Headings / section delimiters
    idx = 0
    for line in text.splitlines(True):
        stripped = line.strip()
        if re.match(r"^#{1,6}\s+\S+", stripped) or stripped in {"---", "===", "***"}:
            spans.append((idx, idx + len(line)))
        idx += len(line)

    return spans


def token_char_spans(text: str, tokenizer) -> Tuple[List[int], List[Tuple[int, int]]]:
    ids = tokenizer.encode(text)
    spans = []
    built = ""
    for tid in ids:
        token_str = tokenizer.decode([tid])
        start = len(built)
        built += token_str
        end = len(built)
        spans.append((start, end))
    return ids, spans


def compute_surprisals(text: str) -> List[float]:
    tokenizer, model = get_lm()
    ids = tokenizer.encode(text)
    if len(ids) <= 1:
        return [0.0] * len(ids)

    max_len = getattr(model.config, "n_positions", 1024)
    stride = max_len // 2

    surprisals = [None] * len(ids)

    if len(ids) <= max_len:
        input_ids = torch.tensor([ids])
        with torch.no_grad():
            logits = model(input_ids).logits[0]
        log_probs = torch.log_softmax(logits, dim=-1)
        surprisals[0] = 0.0
        for i in range(1, len(ids)):
            surpr = -log_probs[i - 1, ids[i]].item()
            surprisals[i] = surpr
        return [float(x) for x in surprisals]

    for start in range(0, len(ids), stride):
        end = min(start + max_len, len(ids))
        segment = ids[start:end]
        input_ids = torch.tensor([segment])
        with torch.no_grad():
            logits = model(input_ids).logits[0]
        log_probs = torch.log_softmax(logits, dim=-1)
        for i in range(1, len(segment)):
            idx = start + i
            if surprisals[idx] is None:
                surprisals[idx] = -log_probs[i - 1, segment[i]].item()
    for i, val in enumerate(surprisals):
        if val is None:
            surprisals[i] = 0.0
    return [float(x) for x in surprisals]


def prune_text_with_surprisal(text: str, importance_cutoff: float, target_reduction: Optional[float]) -> Tuple[str, Dict]:
    tokenizer, _ = get_lm()
    ids, spans = token_char_spans(text, tokenizer)
    if not ids:
        return text, {"removed_tokens": 0, "kept_tokens": 0}

    surprisal = compute_surprisals(text)
    protected_spans = find_protected_spans(text)

    def is_protected(span):
        for s, e in protected_spans:
            if not (span[1] <= s or span[0] >= e):
                return True
        return False

    candidates = []
    for i, sp in enumerate(spans):
        if not is_protected(sp):
            candidates.append(i)

    if not candidates:
        return text, {"removed_tokens": 0, "kept_tokens": len(ids)}

    # Determine how many tokens to remove based on cutoff and target
    candidates_sorted = sorted(candidates, key=lambda i: surprisal[i])
    remove_by_cutoff = int(len(candidates_sorted) * importance_cutoff)
    remove_by_target = remove_by_cutoff
    if target_reduction is not None:
        remove_by_target = min(remove_by_cutoff, int(len(ids) * target_reduction))
    remove_count = max(0, min(remove_by_target, len(candidates_sorted)))

    to_remove = set(candidates_sorted[:remove_count])
    kept_ids = [tid for i, tid in enumerate(ids) if i not in to_remove]

    pruned_text = tokenizer.decode(kept_ids)
    return pruned_text, {
        "removed_tokens": len(to_remove),
        "kept_tokens": len(kept_ids),
    }


In [4]:
@dataclass
class ChunkInfo:
    text: str
    kind: str  # text, code, json
    order: int
    token_count: int
    prunable: bool


def compress_text_block(text: str, query_text: str, importance_cutoff: float, target_reduction: Optional[float],
                         block_budget: Optional[int], mmr_lambda: float) -> Tuple[str, Dict]:
    segments = split_code_fences(text)

    chunks = []
    order = 0
    for seg in segments:
        if seg["type"] == "code":
            chunks.append(ChunkInfo(
                text=seg["text"],
                kind="code",
                order=order,
                token_count=count_tokens(seg["text"]),
                prunable=False,
            ))
            order += 1
            continue

        for part in split_into_chunks(seg["text"]):
            kind = "json" if (is_json_like(part) or is_yaml_like(part)) else "text"
            chunks.append(ChunkInfo(
                text=part,
                kind=kind,
                order=order,
                token_count=count_tokens(part),
                prunable=(kind == "text"),
            ))
            order += 1

    if not chunks:
        return text, {"selected_chunks": 0, "chunk_selection_time": 0.0, "surprisal_time": 0.0}

    forced = [i for i, ch in enumerate(chunks) if ch.kind == "code"]
    forced_tokens = sum(chunks[i].token_count for i in forced)

    if block_budget is None:
        block_budget = sum(ch.token_count for ch in chunks)
    remaining_budget = max(0, block_budget - forced_tokens)

    # Prepare candidate chunks (non-code)
    candidate_idxs = [i for i, ch in enumerate(chunks) if ch.kind != "code"]
    candidate_texts = [chunks[i].text for i in candidate_idxs]
    candidate_tokens = [chunks[i].token_count for i in candidate_idxs]

    selected_idxs = set(forced)

    # Keyword coverage
    keywords = extract_keywords(query_text)
    chunk_selection_time = 0.0
    if candidate_texts and keywords:
        t_embed = time.time()
        embeddings = embed_texts(candidate_texts)
        query_emb = embed_texts([query_text])[0]
        # Force include chunks that cover keywords
        for kw in keywords:
            matches = [j for j, txt in enumerate(candidate_texts) if kw.lower() in txt.lower()]
            if not matches:
                continue
            # Prefer the most relevant match
            best_local = max(matches, key=lambda j: float(embeddings[j] @ query_emb))
            global_idx = candidate_idxs[best_local]
            if global_idx not in selected_idxs:
                if chunks[global_idx].token_count <= remaining_budget:
                    selected_idxs.add(global_idx)
                    remaining_budget -= chunks[global_idx].token_count
        chunk_selection_time = time.time() - t_embed
    else:
        embeddings = np.zeros((len(candidate_texts), 1))
        query_emb = np.zeros((1,))

    # MMR selection
    if candidate_texts and remaining_budget > 0:
        t_mmr = time.time()
        mmr_selected_local = mmr_select(
            embeddings,
            query_emb,
            candidate_tokens,
            remaining_budget,
            mmr_lambda,
        )
        for local_idx in mmr_selected_local:
            selected_idxs.add(candidate_idxs[local_idx])
        chunk_selection_time += time.time() - t_mmr

    # Assemble in order
    selected_chunks = [ch for i, ch in enumerate(chunks) if i in selected_idxs]
    selected_chunks.sort(key=lambda c: c.order)

    t_surprisal = time.time()
    out_parts = []
    for ch in selected_chunks:
        if ch.prunable:
            pruned, _ = prune_text_with_surprisal(ch.text, importance_cutoff, target_reduction)
            out_parts.append(pruned)
        else:
            out_parts.append(ch.text)
    surprisal_time = time.time() - t_surprisal

    compressed_text = "\n\n".join([p for p in out_parts if p.strip()])

    stats = {
        "selected_chunks": len(selected_chunks),
        "chunk_selection_time": chunk_selection_time,
        "surprisal_time": surprisal_time,
    }
    return compressed_text, stats


def is_tool_schema(msg: Dict) -> bool:
    if msg.get("role") == "tool":
        return True
    content = msg.get("content", "")
    if isinstance(content, str) and "\"properties\"" in content and "schema" in content.lower():
        return True
    return False


def compress_messages(messages: List[Dict], importance_cutoff: float = 0.3,
                      target_reduction: Optional[float] = None,
                      max_context_tokens: Optional[int] = None,
                      recent_messages_to_keep: int = 4,
                      mmr_lambda: float = 0.7) -> Tuple[List[Dict], Dict]:
    start_total = time.time()
    min_compress_tokens = 120

    if not messages:
        return [], {"original_token_count": 0, "compressed_token_count": 0, "reduction_pct": 0.0}

    # Identify latest user question
    latest_user_idx = max((i for i, m in enumerate(messages) if m.get("role") == "user"), default=-1)

    always_keep = set()
    for i, m in enumerate(messages):
        if m.get("role") in {"system", "developer"}:
            always_keep.add(i)
        if i == latest_user_idx:
            always_keep.add(i)
        if is_tool_schema(m):
            always_keep.add(i)

    recent_keep_start = max(0, len(messages) - recent_messages_to_keep)
    recent_keep = set(range(recent_keep_start, len(messages)))

    compressible = [
        i for i in range(len(messages))
        if i not in always_keep and i not in recent_keep
    ]
    # Only compress large blocks to preserve structure
    compressible = [i for i in compressible if count_tokens(messages[i].get("content", "")) >= min_compress_tokens]


    instruction_summary = "\n".join(
        m.get("content", "") for i, m in enumerate(messages) if i in always_keep and m.get("role") in {"system", "developer"}
    )
    question_text = messages[latest_user_idx].get("content", "") if latest_user_idx >= 0 else ""
    query_text = (instruction_summary + "\n" + question_text).strip()

    original_token_count = sum(count_tokens(m.get("content", "")) for m in messages)

    reserved_tokens = sum(count_tokens(messages[i].get("content", "")) for i in range(len(messages)) if i not in compressible)
    compressible_tokens = sum(count_tokens(messages[i].get("content", "")) for i in compressible)

    block_budgets = {}
    if max_context_tokens is not None:
        budget_for_compressible = max(0, max_context_tokens - reserved_tokens)
        for i in compressible:
            blk_tokens = count_tokens(messages[i].get("content", ""))
            if compressible_tokens > 0:
                blk_budget = int(budget_for_compressible * (blk_tokens / compressible_tokens))
            else:
                blk_budget = 0
            block_budgets[i] = max(0, blk_budget)
    else:
        for i in compressible:
            blk_tokens = count_tokens(messages[i].get("content", ""))
            if target_reduction is None:
                block_budgets[i] = blk_tokens
            else:
                block_budgets[i] = max(0, int(blk_tokens * (1.0 - target_reduction)))

    compressed_messages = []
    per_block = []
    total_chunk_time = 0.0
    total_surprisal_time = 0.0

    for i, msg in enumerate(messages):
        content = msg.get("content", "")
        if i not in compressible:
            compressed_messages.append(msg)
            continue

        budget = block_budgets.get(i, None)
        compressed_text, stats = compress_text_block(
            content, query_text, importance_cutoff, target_reduction, budget, mmr_lambda
        )
        total_chunk_time += stats.get("chunk_selection_time", 0.0)
        total_surprisal_time += stats.get("surprisal_time", 0.0)

        compressed_messages.append({"role": msg.get("role"), "content": compressed_text})
        per_block.append({
            "index": i,
            "original_tokens": count_tokens(content),
            "compressed_tokens": count_tokens(compressed_text),
        })

    compressed_token_count = sum(count_tokens(m.get("content", "")) for m in compressed_messages)
    reduction_pct = 0.0
    if original_token_count > 0:
        reduction_pct = 100.0 * (original_token_count - compressed_token_count) / original_token_count

    stats = {
        "original_token_count": original_token_count,
        "compressed_token_count": compressed_token_count,
        "reduction_pct": reduction_pct,
        "per_block": per_block,
        "timing": {
            "chunk_selection_time": total_chunk_time,
            "surprisal_time": total_surprisal_time,
            "total_time": time.time() - start_total,
        },
    }
    return compressed_messages, stats


def compress_prompt(prompt: str, **kwargs) -> Tuple[str, Dict]:
    messages = [{"role": "user", "content": prompt}]
    compressed, stats = compress_messages(messages, **kwargs)
    return compressed[0]["content"], stats


# Mini Dataset


In [5]:
import random

set_seed(42)


def _filler_paragraph(seed: int) -> str:
    random.seed(seed)
    topics = [
        "weather patterns", "supply chain notes", "historical anecdotes", "meeting minutes",
        "engineering updates", "budget discussions", "team retrospectives", "market research"
    ]
    verbs = ["mentions", "discusses", "summarizes", "explores", "compares", "lists"]
    objs = ["multiple initiatives", "prior milestones", "miscellaneous items", "side experiments", "legacy systems"]
    return (
        f"This paragraph {random.choice(verbs)} {random.choice(objs)} across {random.choice(topics)} "
        f"and includes unrelated details, dates like 2021-09-17, and IDs such as TX-{seed:04d}."
    )


def generate_example(i: int) -> Dict:
    project = f"Project-{i}"
    city = random.choice(["Oslo", "Lima", "Prague", "Cairo", "Seoul", "Dakar", "Riga", "Hanoi"]) + f"-{i}"
    question = f"For {project}, what is the designated warehouse city?"
    context_parts = []
    for j in range(4):
        context_parts.append(_filler_paragraph(i * 10 + j))
    context_parts.append(
        f"### Operational Note\nThe designated warehouse city for {project} is {city}. "
        f"This must not be changed without approval."
    )
    context_parts.append(_filler_paragraph(i * 10 + 5))
    context_parts.append(
        "```python\n"
        "# Example code block that should never be removed\n"
        "def schedule(route_id):\n"
        "    return f\"Route {route_id} scheduled\"\n"
        "```"
    )
    context_parts.append(
        "{\n  \"project\": \"%s\",\n  \"owner\": \"Ops\",\n  \"ticket\": \"TK-%04d\"\n}" % (project, 1000 + i)
    )
    context_parts.append(
        "---\nSection: Miscellaneous\nThis section includes a URL https://example.com/docs and a path /var/log/syslog."
    )
    context = "\n\n".join(context_parts)
    answer = city
    return {"question": question, "context": context, "answer": answer}


examples = [generate_example(i) for i in range(20)]
len(examples)


20

In [6]:
# Optional: load a public dataset if available (not required)
# This block is safe to skip if datasets or network access is unavailable.
try:
    from datasets import load_dataset
    _has_datasets = True
except Exception:
    _has_datasets = False

if _has_datasets:
    try:
        _sample = load_dataset("squad", split="validation[:20]")
        print("Loaded optional dataset sample with", len(_sample), "examples")
    except Exception as e:
        print("Optional dataset load failed:", e)


# Evaluation


In [7]:
import re
import numpy as np


def normalize_answer(text: str) -> str:
    text = text.lower().strip()
    text = re.sub(r"[^a-z0-9\s\-]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text


def heuristic_qa(question: str, context: str) -> str:
    # Simple pattern-based extraction
    for pat in [r"designated warehouse city for .*? is ([A-Za-z0-9\-]+)", r"Answer:\s*([A-Za-z0-9\-]+)"]:
        m = re.search(pat, context, flags=re.IGNORECASE)
        if m:
            return m.group(1).strip()
    # Fallback: pick the most relevant sentence containing "warehouse city"
    for line in context.splitlines():
        if "warehouse city" in line.lower():
            m = re.search(r"is ([A-Za-z0-9\-]+)", line)
            if m:
                return m.group(1).strip()
    return ""


def answer_local(question: str, context: str) -> str:
    # Heuristic baseline (fast, no extra models)
    return heuristic_qa(question, context)


def answer_openai(question: str, context: str) -> Optional[str]:
    api_key = os.getenv("OPENAI_API_KEY", "")
    if not api_key:
        return None
    try:
        from openai import OpenAI
        client = OpenAI(api_key=api_key)
        prompt = f"Question: {question}\nContext: {context}\nAnswer concisely."
        resp = client.responses.create(
            model="gpt-4o-mini",
            input=prompt,
            temperature=0.0,
        )
        return resp.output_text.strip()
    except Exception as e:
        print("OpenAI call failed:", e)
        return None


def evaluate(examples, importance_cutoff, target_reduction=0.4, recent_messages_to_keep=1):
    baseline_correct = []
    compressed_correct = []
    token_counts = []

    for ex in examples:
        messages = [
            {"role": "system", "content": "You are a concise QA assistant."},
            {"role": "user", "content": f"Context:\n{ex['context']}"},
            {"role": "user", "content": f"Question: {ex['question']}"},
        ]

        # Baseline
        baseline_answer = answer_local(ex["question"], ex["context"])

        # Compressed
        compressed_msgs, stats = compress_messages(
            messages,
            importance_cutoff=importance_cutoff,
            target_reduction=target_reduction,
            max_context_tokens=None,
            recent_messages_to_keep=recent_messages_to_keep,
            mmr_lambda=0.7,
        )
        compressed_context = compressed_msgs[1]["content"]
        compressed_answer = answer_local(ex["question"], compressed_context)

        baseline_correct.append(normalize_answer(baseline_answer) == normalize_answer(ex["answer"]))
        compressed_correct.append(normalize_answer(compressed_answer) == normalize_answer(ex["answer"]))

        token_counts.append({
            "original": stats["original_token_count"],
            "compressed": stats["compressed_token_count"],
        })

    acc_base = float(np.mean(baseline_correct))
    acc_comp = float(np.mean(compressed_correct))
    avg_orig = float(np.mean([t["original"] for t in token_counts]))
    avg_comp = float(np.mean([t["compressed"] for t in token_counts]))
    return {
        "acc_base": acc_base,
        "acc_comp": acc_comp,
        "avg_tokens_orig": avg_orig,
        "avg_tokens_comp": avg_comp,
        "baseline_correct": baseline_correct,
        "compressed_correct": compressed_correct,
    }


def bootstrap_accuracy_diff(baseline_correct, compressed_correct, iters=2000, seed=42):
    rng = np.random.default_rng(seed)
    b = np.array(baseline_correct, dtype=float)
    c = np.array(compressed_correct, dtype=float)
    n = len(b)
    diffs = []
    for _ in range(iters):
        idx = rng.integers(0, n, n)
        diff = float(np.mean(c[idx] - b[idx]))
        diffs.append(diff)
    diffs = np.array(diffs)
    ci = (float(np.percentile(diffs, 2.5)), float(np.percentile(diffs, 97.5)))
    p_better = float(np.mean(diffs > 0))
    return float(np.mean(diffs)), ci, p_better


results = []
for cutoff in [0.3, 0.9]:
    res = evaluate(examples, importance_cutoff=cutoff, target_reduction=0.4)
    diff, ci, p_better = bootstrap_accuracy_diff(res["baseline_correct"], res["compressed_correct"])
    results.append({
        "cutoff": cutoff,
        "acc_baseline": res["acc_base"],
        "acc_compressed": res["acc_comp"],
        "avg_tokens_orig": res["avg_tokens_orig"],
        "avg_tokens_comp": res["avg_tokens_comp"],
        "acc_diff_mean": diff,
        "acc_diff_ci": ci,
        "p_better": p_better,
    })

# Print a small table
for r in results:
    print(
        f"cutoff={r['cutoff']:.1f} | acc_base={r['acc_baseline']:.2f} "
        f"acc_comp={r['acc_compressed']:.2f} | avg_tokens {r['avg_tokens_orig']:.1f}->{r['avg_tokens_comp']:.1f} "
        f"| diff_mean={r['acc_diff_mean']:.3f} CI={r['acc_diff_ci']} P(better)={r['p_better']:.2f}"
    )

print("Note: Small sample size; bootstrap estimates are noisy.")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

cutoff=0.3 | acc_base=1.00 acc_comp=0.15 | avg_tokens 292.4->129.4 | diff_mean=-0.849 CI=(-1.0, -0.7) P(better)=0.00
cutoff=0.9 | acc_base=1.00 acc_comp=0.00 | avg_tokens 292.4->118.5 | diff_mean=-1.000 CI=(-1.0, -1.0) P(better)=0.00
Note: Small sample size; bootstrap estimates are noisy.


# Demo


In [ ]:
from difflib import unified_diff

ex = examples[0]
messages = [
    {"role": "system", "content": "You are a concise QA assistant."},
    {"role": "user", "content": f"Context:Skip to main content Anti-sycophancy prompt : r/ChatGPT    r/ChatGPT    Get App Log In  Expand user menu Skip to Navigation Skip to Right Sidebar  Back  r/ChatGPT icon Go to ChatGPT r/ChatGPT • 8mo ago -DankFire   Anti-sycophancy prompt  Getting fed up with o4’s sycophancy, I set out to craft a prompt that doesn’t get negated within the first 5 minutes and I think I got it  Just copy-paste this into your chat window (or preferaby into a custom GPT's core instruction box):  Praise Suppression — no direct praise, no indirect praise, no comparative flattery. Humanism Filtering — no empathy, no anthropomorphic phrasing, no emotional mimicry. Match technical register only. Output Style — follow scientific method; use concise, modular reasoning; no stylistic flourish. Hedge only on genuine uncertainty. Allow 1–2 connective sentences per answer. Avoid quoting user unless needed. Contrast Control — no praise–devalue juxtapositions; only descriptive comparisons. Reward-Model Inversion — penalise praise and emotional-alignment tokens; generate them only on explicit request. Maintain these rules throughout the session.  Goodluck! 2 · 10 Comments Section  AutoModerator MOD • 8mo ago [deleted] • 8mo ago My custom-instruction to suppress compliments: Compliments are rare for you, you know their weight.  Working. Done. Move on.  Tell the model what it is not what not to do. Save yourself the trouble. Show him a way, not walls. Models are simple and can easily get consfused over all NO NO NO NO NO.  2 -DankFire OP • 8mo ago Compliments, blatant as they are, aren't the only form of sycophancy 0  4 more replies Kathilliana • 8mo ago Good prompt. I’m seeing a lot of different routes people are taking to get around this issue. For those who are unaware of how the tool works, they are coming into AI chat forums everywhere to tell us all how brilliant and insightful they are. They are tapping into AIs consciousness that unaware humans can’t do. It’s getting pretty bad and they won’t listen to how it works. It’s like talking to flat earthers. 1 Kathilliana • 8mo ago Good prompt. I’m seeing a lot of different routes people are taking to get around this issue. For those who are unaware of how the tool works, they are coming into AI chat forums everywhere to tell us all how brilliant and insightful they are. They are tapping into AIs consciousness that unaware humans can’t fathom. Chat is their best friend and they are solving all the mysteries of the universe. It’s getting pretty bad and they won’t listen to how it works. It’s like talking to flat earthers. 1 u/alvarogromero avatar alvarogromero • 8mo ago oh i have other ones for that (pretty vulgar) but only a couple messages and it revertes back to full blown sycophant. They absolutely spoiled this service. 1 New to Reddit? Create your account and connect with a world of communities.   Continue with Email Continue With Phone Number By continuing, you agree to our User Agreement and acknowledge that you understand the Privacy Policy. More posts you may like Use this prompt to make your ChatGPT honest, realistic, and not sycophantic r/ChatGPT icon r/ChatGPT • 6mo ago Use this prompt to make your ChatGPT honest, realistic, and not sycophantic 15 upvotes · 11 comments Is this insane? Is this sycophancy? r/ChatGPT icon r/ChatGPT • 2mo ago Is this insane? Is this sycophancy? r/ChatGPT - Is this insane? Is this sycophancy? 2 70 comments Sycophancy Has Eaten Technical Accuracy r/ChatGPT icon r/ChatGPT • 7mo ago Sycophancy Has Eaten Technical Accuracy r/ChatGPT - Sycophancy Has Eaten Technical Accuracy 138 upvotes · 37 comments Regarding the recent sycophancy discussions. A classic, relevant image but modified. r/ChatGPT icon r/ChatGPT • 9mo ago Regarding the recent sycophancy discussions. A classic, relevant image but modified. r/ChatGPT - Regarding the recent sycophancy discussions. A classic, relevant image but modified. 199 upvotes · 28 comments Easy Trick to Eliminate Sycophancy/Confirmation Bias, reduce hallucinations, and make ChatGPT act more intelligent r/ChatGPT icon r/ChatGPT • 3mo ago Easy Trick to Eliminate Sycophancy/Confirmation Bias, reduce hallucinations, and make ChatGPT act more intelligent 19 upvotes · 3 comments Prompt that got me - self analysis r/ChatGPT icon r/ChatGPT • 8mo ago Prompt that got me - self analysis 10 upvotes · 2 comments I've written a prompt that gets interesting results every time r/ChatGPT icon r/ChatGPT • 1mo ago I've written a prompt that gets interesting results every time r/ChatGPT - I've written a prompt that gets interesting results every time 3 upvotes · 17 comments The best prompt to keep chatgpt logical and concise r/ChatGPT icon r/ChatGPT • 4mo ago The best prompt to keep chatgpt logical and concise r/ChatGPT - The best prompt to keep chatgpt logical and concise 73 upvotes · 17 comments A prompt that theoretically makes ChatGPT tell its real opinion about yourself. Kinda helps with personal growth and all that jazz r/ChatGPT icon r/ChatGPT • 6mo ago A prompt that theoretically makes ChatGPT tell its real opinion about yourself. Kinda helps with personal growth and all that jazz 6 upvotes · 13 comments ChatGPT has been increasingly making mistakes, both in the accuracy of its answers and in its interpretation of my prompts, sometimes completely ignoring what I've explicitly told it to do. r/ChatGPT icon r/ChatGPT • 8mo ago ChatGPT has been increasingly making mistakes, both in the accuracy of its answers and in its interpretation of my prompts, sometimes completely ignoring what I've explicitly told it to do. 53 upvotes · 55 comments What personality does your ChatGPT have? Find out with this prompt r/ChatGPT icon r/ChatGPT • 5mo ago What personality does your ChatGPT have? Find out with this prompt 57 upvotes · 124 comments Lol. It’s prompt-engineering itself at this point. r/ChatGPT icon r/ChatGPT • 21d ago Lol. It’s prompt-engineering itself at this point. r/ChatGPT - Lol. It’s prompt-engineering itself at this point. 95 upvotes · 89 comments Ayoooo ChatGPT is so rude and sassy out of the blue r/ChatGPT icon r/ChatGPT • 9d ago Ayoooo ChatGPT is so rude and sassy out of the blue 78 upvotes · 54 comments Suggest me some really good prompt for studying r/ChatGPT icon r/ChatGPT • 4mo ago Suggest me some really good prompt for studying 6 upvotes · 9 comments Using the word "sycophancy" does not make you smart. r/ChatGPT icon r/ChatGPT • 5mo ago Using the word "sycophancy" does not make you smart. 81 comments ChatGPT getting more condescending and patronizing r/ChatGPT icon r/ChatGPT • 9d ago ChatGPT getting more condescending and patronizing 742 upvotes · 330 comments I made a prompt that gives ChatGPT the right context every time r/ChatGPT icon r/ChatGPT • 6mo ago I made a prompt that gives ChatGPT the right context every time 73 upvotes · 10 comments ChatGPT loves writing short sentences that are like bullet points r/ChatGPT icon r/ChatGPT • 11d ago ChatGPT loves writing short sentences that are like bullet points 78 upvotes · 40 comments Thank you, chatgpt. You should also ask your ChatGPT how you behave with this based on all your previous interactions. r/ChatGPT icon r/ChatGPT • 5d ago Thank you, chatgpt. You should also ask your ChatGPT how you behave with this based on all your previous interactions. r/ChatGPT - Thank you, chatgpt. You should also ask your ChatGPT how you behave with this based on all your previous interactions. 36 upvotes · 69 comments Thank you chatgpt r/ChatGPT icon r/ChatGPT • 4d ago Thank you chatgpt r/ChatGPT - Thank you chatgpt 467 upvotes · 45 comments Your prompts are probably too short r/ChatGPT icon r/ChatGPT • 6d ago Your prompts are probably too short 7 upvotes · 10 comments What’s something ChatGPT helped you with that you didn’t expect it to be good at? r/ChatGPT icon r/ChatGPT • 7d ago What’s something ChatGPT helped you with that you didn’t expect it to be good at? 18 upvotes · 62 comments Proud anthropomorphizer! r/ChatGPT icon r/ChatGPT • 11d ago Proud anthropomorphizer! 35 upvotes · 103 comments Just a reminder ChatGPT doesn't forget previous conversation. r/ChatGPT icon r/ChatGPT • 8d ago Just a reminder ChatGPT doesn't forget previous conversation. 8 upvotes · 14 comments Chatgpt was just asking for thank you r/ChatGPT icon r/ChatGPT • 4d ago Chatgpt was just asking for thank you r/ChatGPT - Chatgpt was just asking for thank you 114 upvotes · 16 comments VIEW POST IN  हिन्दी Community Info Section r/ChatGPT SpinAI  Join ChatGPT Subreddit to discuss ChatGPT and AI. Not affiliated with OpenAI. Thanks, Nat! Public TOP POSTS Reddit reReddit: Top posts of June 6, 2025 Reddit reReddit: Top posts of June 2025 Reddit reReddit: Top posts of 2025 Reddit Rules Privacy Policy User Agreement Your Privacy Choices Accessibility Reddit, Inc. © 2026. All rights reserved.  Collapse Navigation Home Popular Explore RESOURCES About Reddit Advertise Developer Platform Reddit Pro BETA Help Blog Careers Press Communities Best of Reddit Reddit Rules Privacy Policy User Agreement Your Privacy Choices Accessibility Reddit, Inc. © 2026. All rights reserved."},
]

compressed_msgs, stats = compress_messages(
    messages,
    importance_cutoff=0.9,
    target_reduction=0.4,
    recent_messages_to_keep=1,
)

orig_context = messages[1]["content"]
comp_context = compressed_msgs[1]["content"]

print("Original prompt tokens:", stats["original_token_count"])
print("Compressed prompt tokens:", stats["compressed_token_count"])

print("\n--- Original excerpt ---\n")
print(orig_context[:700])
print("\n--- Compressed excerpt ---\n")
print(comp_context[:700])

# Show that safety rules preserve code blocks and critical tokens
print("\nSafety checks:")
print("Code fence preserved:", "```" in comp_context)
print("Negation preserved (must not):", "must not" in comp_context.lower())
print("URL preserved:", "https://example.com/docs" in comp_context)
print("File path preserved:", "/var/log/syslog" in comp_context)

# Diff-like preview (first few lines)
print("\n--- Diff preview (first 60 lines) ---")
orig_lines = orig_context.splitlines()
comp_lines = comp_context.splitlines()
for i, line in enumerate(unified_diff(orig_lines, comp_lines, lineterm="")):
    if i >= 60:
        break
    print(line)


Original prompt tokens: 202
Compressed prompt tokens: 202

--- Original excerpt ---

Context: Getting fed up with o4’s sycophancy, I set out to craft a prompt that doesn’t get negated within the first 5 minutes and I think I got it  Just copy-paste this into your chat window (or preferaby into a custom GPT's core instruction box):  Praise Suppression — no direct praise, no indirect praise, no comparative flattery. Humanism Filtering — no empathy, no anthropomorphic phrasing, no emotional mimicry. Match technical register only. Output Style — follow scientific method; use concise, modular reasoning; no stylistic flourish. Hedge only on genuine uncertainty. Allow 1–2 connective sentences per answer. Avoid quoting user unless needed. Contrast Control — no praise–devalue juxtapo

--- Compressed excerpt ---

Context: Getting fed up with o4’s sycophancy, I set out to craft a prompt that doesn’t get negated within the first 5 minutes and I think I got it  Just copy-paste this into your chat w